# Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,72.63,80,34,17.81,scattered clouds
1,1,Albany,US,42.60,-73.97,30.99,81,0,1.86,light rain
2,2,Bluff,NZ,-46.60,168.33,59.58,57,0,4.25,clear sky
3,3,Tottori,JP,35.50,134.23,53.60,58,40,11.41,scattered clouds
4,4,Terney,RU,45.05,136.61,32.76,70,0,10.29,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kavieng,PG,-2.57,150.80,86.13,68,100,5.57,light rain
16,16,Itarema,BR,-2.92,-39.92,80.53,79,60,12.33,broken clouds
19,19,Hilo,US,19.73,-155.09,84.20,66,40,9.17,scattered clouds
30,30,The Valley,AI,18.22,-63.06,82.40,69,20,10.29,few clouds
33,33,Almeirim,BR,-1.52,-52.58,77.38,87,100,2.93,overcast clouds
37,37,Avarua,CK,-21.21,-159.78,75.20,83,100,9.17,overcast clouds
39,39,Port Moresby,PG,-9.44,147.18,82.40,79,100,11.56,light rain
40,40,Carnarvon,AU,-24.87,113.63,82.40,32,0,17.22,clear sky
43,43,Mataura,NZ,-46.19,168.86,77.00,41,4,6.53,clear sky
46,46,Bengkulu,ID,-3.80,102.27,82.54,68,68,2.80,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                176
City                   176
Country                176
Lat                    176
Lng                    176
Max Temp               176
Humidity               176
Cloudiness             176
Wind Speed             176
Current Description    176
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
     # Done in step #2 with the'dropna()'

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kavieng,PG,86.13,light rain,-2.57,150.80,
16,Itarema,BR,80.53,broken clouds,-2.92,-39.92,
19,Hilo,US,84.20,scattered clouds,19.73,-155.09,
30,The Valley,AI,82.40,few clouds,18.22,-63.06,
33,Almeirim,BR,77.38,overcast clouds,-1.52,-52.58,
37,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,
39,Port Moresby,PG,82.40,light rain,-9.44,147.18,
40,Carnarvon,AU,82.40,clear sky,-24.87,113.63,
43,Mataura,NZ,77.00,clear sky,-46.19,168.86,
46,Bengkulu,ID,82.54,broken clouds,-3.80,102.27,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"       
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kavieng,PG,86.13,light rain,-2.57,150.80,Nusa Island Retreat
16,Itarema,BR,80.53,broken clouds,-2.92,-39.92,Windvillage
19,Hilo,US,84.20,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
30,The Valley,AI,82.40,few clouds,18.22,-63.06,CeBlue Villas & Beach Resort
33,Almeirim,BR,77.38,overcast clouds,-1.52,-52.58,Ricácio Hotel
...,...,...,...,...,...,...,...
668,Acapulco,MX,82.40,clear sky,16.86,-99.89,HS Hotsson Smart Acapulco
669,Rodrigues Alves,BR,77.52,overcast clouds,-7.74,-72.65,Pousada Aconchego
673,Mayumba,GA,77.95,light rain,-3.43,10.66,LIKWALE LODGE
674,Riohacha,CO,79.16,overcast clouds,11.54,-72.91,Hotel Taroa


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
current_weather = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=current_weather,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))